Starter.py content

In [1]:
import sys

import pylab as plb
import numpy as np
import mountaincar

In [6]:
# FROM Project 1!

# To calculate the gaussian
def gauss(x,p):
    """Return the gauss function N(x), with mean p[0] and std p[1].
    Normalized such that N(x=p[0]) = 1.
    """
    return np.exp((-(x - p[0])**2) / (2 * p[1]**2))

# To calculate the distance between two vectors
def distance(vec1,vec2):
    assert np.shape(vec1)==np.shape(vec2)
    #print("   Distance between",vec1,"and",vec2)
    term1 = np.power(vec1-vec2,2)
    #print("      term1 :",term1)
    term2 = np.sqrt(np.sum(term1,axis=1))
    #print("      term2 :",term2)
    return term2

# To assign data to the closest center
def assign_cluster(centers, datas):
    #initialization
    cluster_assignment = np.zeros(np.shape(datas)[0]) - 1
    current_c = np.tile(centers[0], (np.shape(datas)[0],1))
    eucl_distance = distance(current_c,datas)
    min_distance = np.copy(eucl_distance)
    #find cluster for each entry in datas
    for i in range(0,np.shape(centers)[0]):
        #print("Center is : ",centers[i], "iteration : ",i)
        current_c = np.tile(centers[i], (np.shape(datas)[0],1))
        eucl_distance = distance(current_c,datas)
        #print("   MD :",min_distance)
        #print("   ED :",eucl_distance)
        bool_distance = eucl_distance <= min_distance
        #print("   BD :",bool_distance)
        min_distance = eucl_distance*(bool_distance) + min_distance*(1 - bool_distance)
        #print("   MD :",min_distance)        
        cluster_assignment = cluster_assignment*(1 - bool_distance) + i*bool_distance
        #print("   CA :",cluster_assignment)
    return cluster_assignment

In [75]:
class DummyAgent():
    """A not so good agent for the mountain-car task.
    """

    def __init__(self, mountain_car = None, parameter1 = 3.0):
        
        if mountain_car is None:
            self.mountain_car = mountaincar.MountainCar()
        else:
            self.mountain_car = mountain_car

        self.parameter1 = parameter1

    def visualize_trial(self, n_steps = 200):
        """Do a trial without learning, with display.

        Parameters
        ----------
        n_steps -- number of steps to simulate for
        """
        
        # prepare for the visualization
        plb.ion()
        mv = mountaincar.MountainCarViewer(self.mountain_car)
        mv.create_figure(n_steps, n_steps)
        plb.draw()
            
        # make sure the mountain-car is reset
        self.mountain_car.reset()

        for n in range(n_steps):
            print ('\rt =', self.mountain_car.t,
            sys.stdout.flush())
            
            # choose a random action
            self.mountain_car.apply_force(np.random.randint(3) - 1)
            # simulate the timestep
            self.mountain_car.simulate_timesteps(100, 0.01) #MEANS simulation EVERY second of the state of the car !!!

            # update the visualization
            mv.update_figure()
            plb.draw()            
            
            # check for rewards
            if self.mountain_car.R > 0.0:
                print ("\rreward obtained at t = ", self.mountain_car.t)
                break


    def learn(self):
        # This is your job!
        
        # Implement Sarsa Lambda: Reinforcement Learning Algo
        # TO DO: http://www.cse.unsw.edu.au/~cs9417ml/RL1/algorithms.html
        #
        # Initialize Q(s,a) arbitrarily
        # Repeat (for each peisode)
        #   Initialize s
        #   Choose a from s using policy derived from Q (e.g. Espylon greedy)
        #   Repeat (for each step of episode)
        #     Take action a, observe r, s'
        #     Choose a' from s' using policy derived from Q (e.g. Epsylon greedy)
        #     Q(s,a) <-- Q(s,a) + Alpha(r + Gamma*Q(s',a') - Q(s,a));
        #     s <-- s';      a <-- a';
        #   until s is terminal
        
        
        #STEP 1 : Grid Creation !
        # Initialization of the neuron grid size
        x_min=-150;
        x_max=30;
        xPoint_min=-15;
        xPoint_max=15;
        
        # Number of Neurons
        nNeurons=20;
        dimNeurons=2; #position and velocity

        #Initialization of centers (first one of the tab is bottom left, and goes left to right an then down to top): bottom left to up right
        pos=np.linspace(x_min,x_max,nNeurons);
        pos=np.tile(pos,nNeurons);
        vel=np.linspace(xPoint_min,xPoint_max,nNeurons);
        vel=np.tile(vel,nNeurons);
        vel=sorted(vel);

        centers = np.ones((nNeurons**2, dimNeurons)); #DIM of centers --> 400x2
        centers[:,0]=centers[:,0]*pos;
        centers[:,1]=centers[:,1]*vel;

        #STEP 2 : Allocating each time step to a state within the grid !
        
        #STEP 3 : Compute Activity of each neuron within the grid !
        
        
        
        # Q(s,a) = sum(w_aj*r_j(s),j)
        # w_aj = ???
        # r_j(s)=exp(- ((x_j-x)/Sigma_x)^2
        #            - ((xPoint_j-xPoint)/Sigma_xPoint)^2)
        #           )   
        
        # activity of neurons (see project description)
#        for s: #state is made of position and speed (x and xPoint)
#            for j: # each neuron of the map
#                r(s,j)=exp()
        
        
        pass

In [101]:
#TEST TO GET RIGHT THE CENTERS OF THE GRID !!!

nNeurons=20;
dimNeurons=2; #position and velocity

pos=np.linspace(-150,30,20);
pos=np.tile(pos,20);
vel=np.linspace(-15,15,20);
vel=np.tile(vel,20);
vel=sorted(vel);

centers = np.ones((nNeurons**2, dimNeurons));
centers[:,0]=centers[:,0]*pos;
centers[:,1]=centers[:,1]*vel;

centers


array([[-150.        ,  -15.        ],
       [-140.52631579,  -15.        ],
       [-131.05263158,  -15.        ],
       [-121.57894737,  -15.        ],
       [-112.10526316,  -15.        ],
       [-102.63157895,  -15.        ],
       [ -93.15789474,  -15.        ],
       [ -83.68421053,  -15.        ],
       [ -74.21052632,  -15.        ],
       [ -64.73684211,  -15.        ],
       [ -55.26315789,  -15.        ],
       [ -45.78947368,  -15.        ],
       [ -36.31578947,  -15.        ],
       [ -26.84210526,  -15.        ],
       [ -17.36842105,  -15.        ],
       [  -7.89473684,  -15.        ],
       [   1.57894737,  -15.        ],
       [  11.05263158,  -15.        ],
       [  20.52631579,  -15.        ],
       [  30.        ,  -15.        ],
       [-150.        ,  -13.42105263],
       [-140.52631579,  -13.42105263],
       [-131.05263158,  -13.42105263],
       [-121.57894737,  -13.42105263],
       [-112.10526316,  -13.42105263],
       [-102.63157895,  -

In [93]:
d = DummyAgent()
d.visualize_trial()
plb.show()

t = 0.0 None
t = 1.0 None
t = 2.0 None
t = 3.0 None
t = 4.0 None
t = 5.0 None
t = 6.0 None
t = 7.0 None
t = 8.0 None
t = 9.0 None
t = 10.0 None
t = 11.0 None
t = 12.0 None
t = 13.0 None
t = 14.0 None
t = 15.0 None
t = 16.0 None
t = 17.0 None
t = 18.0 None
t = 19.0 None
t = 20.0 None
t = 21.0 None
t = 22.0 None
t = 23.0 None
t = 24.0 None
t = 25.0 None
t = 26.0 None
t = 27.0 None
t = 28.0 None
t = 29.0 None
t = 30.0 None
t = 31.0 None
t = 32.0 None
t = 33.0 None
t = 34.0 None
t = 35.0 None
t = 36.0 None
t = 37.0 None
t = 38.0 None
t = 39.0 None
t = 40.0 None
t = 41.0 None
t = 42.0 None
t = 43.0 None
t = 44.0 None
t = 45.0 None
t = 46.0 None
t = 47.0 None
t = 48.0 None
t = 49.0 None
t = 50.0 None
t = 51.0 None
t = 52.0 None
t = 53.0 None
t = 54.0 None
t = 55.0 None
t = 56.0 None
t = 57.0 None
t = 58.0 None
t = 59.0 None
t = 60.0 None
t = 61.0 None
t = 62.0 None
t = 63.0 None
t = 64.0 None
t = 65.0 None
t = 66.0 None
t = 67.0 None
t = 68.0 None
t = 69.0 None
t = 70.0 None
t = 71.0 None
t 